# Ollama Finetuning
* transformers fix: pip install transformers==2.4.1

In [84]:
from llama_index.llms.ollama import Ollama
from llama_index.core.bridge.pydantic import BaseModel
from loguru import logger

## config

In [85]:
class configLLM:
    model_name = "llama3.1:latest"
    request_timeout = 240.0
    json_mode= True
    context_window = 8000

## set LLM

In [86]:
params = configLLM()

llm = Ollama(
    model=params.model_name,
    request_timeout=params.request_timeout,
    context_window=params.context_window,
    json_mode=params.json_mode
)

## structured output

In [87]:
from pydantic import BaseModel, Field

class StructuredOutput(BaseModel):
    """
    Example of structured output
    """
    answer: str = Field(..., 
                        description="The answer to the question.")
    confidence: float = Field(...,
                            description="Confidence value between 0-1 of the correctness of the result.",
    )
    confidence_explanation: str = Field(
        ..., description="Explanation for the confidence score"
    )
    sources: str = Field(
        ..., description="Sources used to generate the answer, if any."
    )
    model_name: str = Field(
        ..., description="Name of the model used to generate the answer."
    )

structured_llm = llm.as_structured_llm(StructuredOutput)

In [88]:
resp = structured_llm.complete("What is Apple?")

In [89]:
import json

str_resp = resp.model_dump_json()

In [90]:
from docx import Document
from docx.shared import Pt

# convert to dict
dict_resp = json.loads(str_resp)
logger.info(f"Response: {dict_resp}")

2025-07-14 17:17:24.967 | INFO     | __main__:<module>:6 - Response: {'text': '{"answer":"Apple can refer to different things, but here are some of the most common interpretations: } * The fruit: An apple is a sweet and juicy type of fruit that grows on trees ( genus Malus). * Apple Inc.: A multinational technology company that designs, manufactures, and markets consumer electronics, computer software, and online services. Apple is known for its iconic products such as iPhones, iPads, Macs, iPods, iTunes, Safari, iCloud, and more. The company was founded on April 1, 1976, by Steve Jobs, Steve Wozniak, and Ronald Wayne in Cupertino, California. * Apple Records: A British record label that was founded in 1968 by the Beatles and their manager Brian Epstein. The label released music from various artists, including the Beatles themselves, John Lennon, Paul McCartney, George Harrison, and Ringo Starr. * Apple Corps: A multimedia company created in 2007 by Apple Inc. to manage the Beatles\' m

In [91]:
text = dict_resp["text"]
# fom string to dict
text_dict = json.loads(text)

In [ ]:
def create_document(text_dict, filename='llm_response.docx'):
    """
    Create a Word document with the structured response from the LLM.
    """

    doc = Document()

    doc.add_heading('LLM structured Response', level=1)

    doc.add_heading('Answer', level=2)
    doc.add_paragraph(f"{text_dict['answer']}")

    doc.add_heading('LLM Confidence Score', level=2)
    doc.add_paragraph(f"{text_dict['confidence']}")

    doc.add_heading('LLM Confidence Explanation', level=2)
    doc.add_paragraph(f"{text_dict['confidence_explanation']}")

    doc.add_heading('LLM Sources', level=2)
    if text_dict.get('sources'):
        doc.add_paragraph(f"{text_dict['sources']}")
    else:
        doc.add_paragraph("No sources provided.")

    doc.add_heading('LLM Model Name', level=2)
    doc.add_paragraph(f"{text_dict['model_name']}")

    doc.add_heading('Level of Satisfaction with the Answer', level=2)
    table = doc.add_table(rows=1, cols=5)
    table.style = 'Table Grid'
    table.autofit = False
    table.allow_autofit = False
    for row in table.rows:
        for cell in row.cells:
            cell.width = Pt(100)
    table.cell(0, 0).text = '0%'
    table.cell(0, 1).text = '25%'
    table.cell(0, 2).text = '50%'
    table.cell(0, 3).text = '75%'
    table.cell(0, 4).text = '100%'

    # save the document overwrite
    doc.save(filename)

create_document(text_dict)

TypeError: Document.add_heading() got an unexpected keyword argument 'center'